<a href="https://colab.research.google.com/github/wlans01/2021_cphys2/blob/main/final_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Titanic (Regression)

In [40]:
import numpy
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [41]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [42]:
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [43]:
print(train_df.columns.values)

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']


In [44]:
# Survived가 생존여부

# Pclass(객실등급) Sex(성별) Age(나이) sibsp(동반인수) parch(동반인수2) embarked(승선항) 6개의 변수

In [45]:
# 사용하지 않는 변수 제거
train_df = train_df.drop(['Ticket', 'Cabin','Name','PassengerId','Fare'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin','Name','Fare'], axis=1)

In [46]:
# male = 0 female = 1 변경
train_df['Sex']=train_df['Sex'].replace("male",0)
train_df['Sex']=train_df['Sex'].replace("female",0)
test_df['Sex']=test_df['Sex'].replace("male",0)
test_df['Sex']=test_df['Sex'].replace("female",0)

In [47]:
# 승선항 c=0 ,Q=1, s=2
train_df['Embarked'] = train_df['Embarked'].replace('C',0)
train_df['Embarked'] = train_df['Embarked'].replace('Q',1)
train_df['Embarked'] = train_df['Embarked'].replace('S',2)
test_df['Embarked'] = test_df['Embarked'].replace('C',0)
test_df['Embarked'] = test_df['Embarked'].replace('Q',1)
test_df['Embarked'] = test_df['Embarked'].replace('S',2)

In [48]:
# Age 와 Embbarked의 null값을 해결해야함
train_df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Embarked      2
dtype: int64

In [49]:
# 평균값으로 null 채우기
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].mean()) 
test_df['Age'] = test_df['Age'].fillna(test_df['Age'].mean()) 
train_df['Embarked'] = train_df['Embarked'].fillna(train_df['Age'].mean()) 
test_df['Embarked'] = test_df['Embarked'].fillna(test_df['Age'].mean()) 

In [50]:
# 나이 그룹화하기
train_df.loc[ train_df['Age'] <= 10, 'Age_group'] = 0
train_df.loc[(train_df['Age'] > 10) & (train_df['Age'] <= 16), 'Age_group'] = 1
train_df.loc[(train_df['Age'] > 16) & (train_df['Age'] <= 20), 'Age_group'] = 2
train_df.loc[(train_df['Age'] > 20) & (train_df['Age'] <= 26), 'Age_group'] = 3
train_df.loc[(train_df['Age'] > 26) & (train_df['Age'] <= 30), 'Age_group'] = 4
train_df.loc[(train_df['Age'] > 30) & (train_df['Age'] <= 36), 'Age_group'] = 5
train_df.loc[(train_df['Age'] > 36) & (train_df['Age'] <= 40), 'Age_group'] = 6
train_df.loc[(train_df['Age'] > 40) & (train_df['Age'] <= 46), 'Age_group'] = 7
train_df.loc[(train_df['Age'] > 46) & (train_df['Age'] <= 50), 'Age_group'] = 8
train_df.loc[(train_df['Age'] > 50) & (train_df['Age'] <= 60), 'Age_group'] = 9
train_df.loc[ train_df['Age'] > 60, 'Age_group'] = 10

test_df.loc[ test_df['Age'] <= 10, 'Age_group'] = 0
test_df.loc[(test_df['Age'] > 10) & (test_df['Age'] <= 16), 'Age_group'] = 1
test_df.loc[(test_df['Age'] > 16) & (test_df['Age'] <= 20), 'Age_group'] = 2
test_df.loc[(test_df['Age'] > 20) & (test_df['Age'] <= 26), 'Age_group'] = 3
test_df.loc[(test_df['Age'] > 26) & (test_df['Age'] <= 30), 'Age_group'] = 4
test_df.loc[(test_df['Age'] > 30) & (test_df['Age'] <= 36), 'Age_group'] = 5
test_df.loc[(test_df['Age'] > 36) & (test_df['Age'] <= 40), 'Age_group'] = 6
test_df.loc[(test_df['Age'] > 40) & (test_df['Age'] <= 46), 'Age_group'] = 7
test_df.loc[(test_df['Age'] > 46) & (test_df['Age'] <= 50), 'Age_group'] = 8
test_df.loc[(test_df['Age'] > 50) & (test_df['Age'] <= 60), 'Age_group'] = 9
test_df.loc[ test_df['Age'] > 60, 'Age_group'] = 10

In [51]:
# 사용하지 않는 Age 제거
train_df = train_df.drop(['Age'], axis=1)
test_df = test_df.drop(['Age'], axis=1)

In [52]:
# 전처리가 완료된 형태
train_df.shape, test_df.shape

((891, 7), (418, 7))

In [53]:
train_df.isnull().sum()

Survived     0
Pclass       0
Sex          0
SibSp        0
Parch        0
Embarked     0
Age_group    0
dtype: int64

In [54]:
train_df.head(5)

,Survived,Pclass,Sex,SibSp,Parch,Embarked,Age_group
0,0,3,0,1,0,2.0,3.0
1,1,1,0,1,0,0.0,6.0
2,1,3,0,0,0,2.0,3.0
3,1,1,0,1,0,2.0,5.0
4,0,3,0,0,0,2.0,5.0


In [55]:
test_df.head(5)

,PassengerId,Pclass,Sex,SibSp,Parch,Embarked,Age_group
0,892,3,0,0,0,1,5.0
1,893,3,0,1,0,2,8.0
2,894,2,0,0,0,1,10.0
3,895,3,0,0,0,2,4.0
4,896,3,0,1,1,2,3.0


In [ ]:
from sklearn.ensemble import RandomForestClassifier

y = train_df["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch","Age_group","Embarked"]
X = pd.get_dummies(train_df[features])
X_test = pd.get_dummies(test_df[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_df.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")
